# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [105]:
from typing import Tuple, List

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove
from copy import deepcopy



Wielkość planszy

In [106]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [107]:
class Max(Player):
    def __init__(self, char: str) -> None:
        super().__init__(char)

class Min(Player):
    def __init__(self, char: str) -> None:
        super().__init__(char)

In [135]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        if type(game.first_player) is not Max and type(game.first_player) is not Min:
            raise ValueError("The first player is not Max nor Min")
        if type(game.second_player) is not Max and type(game.second_player) is not Min:
            raise ValueError("The second player is not Max nor Min")
        if type(game.first_player) is type(game.second_player):
            raise ValueError("Both players are the same type; One should be Max and one should be Min")
        self.game = game

    def evaluate_position(self, player: Player)->float:
        if self.game.is_finished():
            winner = self.game.get_winner()
            if type(winner) is Max:
                return 1000.0
            elif type(winner) is Min:
                return -1000.0
            else:
                return 0.0
        else:
            pass
            
    def get_best_move(self)->int:
        return self.minimax(10, -100.0, 100.0, type(self.game.state.get_current_player()) is Max)[0]
        
        

    def is_valid_move(self, col_index:int)->bool:
        return not self.game.is_finished() and self.game.state.fields[col_index][ROW_COUNT - 1] is None
    
    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
        valid_moves = self.get_valid_moves()
        if len(valid_moves) == 0 or depth == 0:
            winner = self.game.get_winner()
            if type(winner) is Max:
                return None, 1000.0
            elif type(winner) is Min:
                return None, -1000.0
            elif winner:
                return None, 0.0
            else:
                #Heuristic
                return None, 0.0
            
        else:
            best_move = valid_moves[0]
            if is_maximizing_player:
                evaluation = -1000.1
                for move in valid_moves:
                    evaluated_state = deepcopy(self)
                    evaluated_state.game.state.make_move(ConnectFourMove(move))
                    new_evaluation = evaluated_state.minimax(depth-1, alpha, beta, not is_maximizing_player)[1]
                    if new_evaluation > evaluation:
                        evaluation = new_evaluation
                        best_move = move
                        alpha = max(alpha, evaluation)
                    if alpha >= beta:
                        return best_move, alpha
                return best_move, alpha
            else:
                evaluation = 1000.1
                for move in valid_moves:
                    evaluated_state = deepcopy(self)
                    evaluated_state.game.state.make_move(ConnectFourMove(move))
                    new_evaluation = evaluated_state.minimax(depth-1, alpha, beta, not is_maximizing_player)[1]
                    if new_evaluation < evaluation:
                        evaluation = new_evaluation
                        best_move = move
                        beta = min(beta, evaluation)
                    if alpha >= beta:
                        return best_move, beta
                return best_move, beta

        
    
    def make_move(self, move: int, depth: int):
        move, evaluation = self.minimax(depth, 0.0, 0.0, type(self.game.get_current_player()) is Max)
        self.game.make_move(ConnectFourMove(move))
        print(evaluation)
            
    def get_valid_moves(self) -> list:
        valid_moves = []
        for column in range(len(self.game.state.fields)):
            if self.is_valid_move(column):
                valid_moves.append(column)
        return valid_moves

Rozgrywka

In [136]:
p1 = Max("x")
p2 = Min("o")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
game.make_move(ConnectFourMove(3))

print(game.state.fields[3][ROW_COUNT - 1] is None)


print(type(p1) is Max)

True
True


In [137]:
a = [True, False]
sum(a)


1

In [138]:
p1 = Max("a")
p2 = Min("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)
solver.minimax(4, -1000.1, 1000.1, True)


(0, 0.0)

In [100]:
not False

True